In [1]:
import numpy as np
import pandas as pd 
import requests as re
import time 
import datetime
import requests
from bs4 import BeautifulSoup

# 整理西元日期用於上市/民國用於上櫃（當日資料）

In [23]:
# 整理今日日期 用於上櫃資料（民國日期）

today = datetime.date.today()
year = str(int(str(today)[0:4])-1911)
month = str(today)[5:7]
day = str(today)[8:10]
todate = year + '/' + month +'/' + day
print(todate)

# 西元日期
cdate = str(today)[0:4]+str(today)[5:7]+str(today)[8:10] 
print(cdate)

#如超過日期可進行輸入
#cdate = '20210518'

110/05/19
20210519


# 上市

In [3]:
# 上市 外資投信的網頁
sitca="https://www.twse.com.tw/fund/TWT44U?response=html&date=%s"%cdate
foreign="https://www.twse.com.tw/fund/TWT38U?response=html&date=%s"%cdate
b='b'
s='s'

In [4]:
# 外資投信上市
def tw(sf, bors):
    data2  = pd.read_html(sf)
    df2 = data2[0]
    
    if "44" in sf:
        need2 = [df2.columns[1], df2.columns[2], df2.columns[5]]
        alldata = df2[need2] #把資料換成dataframe
                
        if bors == "s":
            alldata.columns = ['代號','名稱','投信賣超']
        else:
            alldata.columns = ['代號','名稱','投信買超']
            
    else:
        need2 = [df2.columns[1], df2.columns[2], df2.columns[11]]
        alldata = df2[need2] #把資料換成dataframe
                
        if bors =="s":
            alldata.columns = ['代號','名稱','外資賣超']
            for i in range(len(alldata.index)):
                if len(alldata["代號"][i])>4:
                    alldata=alldata.drop(index=i)
        else:
            alldata.columns = ['代號','名稱','外資買超']
            for i in range(len(alldata.index)):
                if len(alldata["代號"][i])>4:
                    alldata=alldata.drop(index=i)
            
    alldata = alldata.reset_index(drop=True)
    
    
    if bors=="s":
        inds = alldata.loc[alldata.iloc[:,2]<0].index    
        selldata = alldata.iloc[inds,:].reset_index(drop =True) #外資淨賣超
        return selldata
    
    else:
        indb = alldata.loc[alldata.iloc[:,2]>0].index #取得index的數字再放到loc裡
        buydata = alldata.iloc[indb,:].reset_index(drop = True) #外資淨買超
        return buydata



In [5]:
total_buy = pd.merge(tw(foreign,b),tw(sitca,b),on=['代號','名稱'],how='outer')
total_sell = pd.merge(tw(foreign,s),tw(sitca,s),on=['代號','名稱'],how='outer')

In [6]:
total = pd.merge(total_buy,total_sell,on=['代號','名稱'], how= 'outer')
for i in range(len(total)):
    if len(total['代號'][i])>4:
        total = total.drop(index=i)
total

,代號,名稱,外資買超,投信買超,外資賣超,投信賣超
0,3481,群創,76115290.0,NaN,NaN,-352000.0
1,2303,聯電,37484751.0,6290000.0,NaN,NaN
2,2344,華邦電,33156420.0,NaN,NaN,NaN
3,2609,陽明,24872810.0,100000.0,NaN,NaN
4,2409,友達,21746200.0,NaN,NaN,-30275000.0
...,...,...,...,...,...,...
923,5533,皇鼎,NaN,NaN,-1000.0,NaN
924,8411,福貞-KY,NaN,NaN,-1000.0,NaN
925,8463,潤泰材,NaN,NaN,-1000.0,NaN
926,8480,泰昇-KY,NaN,NaN,-1000.0,NaN


# 取得同步買超,一買一賣,同步賣超

## 同步買超

In [7]:
total[(total['外資買超']>0) &(total['投信買超']>0)]

,代號,名稱,外資買超,投信買超,外資賣超,投信賣超
1,2303,聯電,37484751.0,6290000.0,NaN,NaN
3,2609,陽明,24872810.0,100000.0,NaN,NaN
5,2882,國泰金,19368300.0,2368000.0,NaN,NaN
6,2317,鴻海,16504010.0,141000.0,NaN,NaN
7,2881,富邦金,16208676.0,2285000.0,NaN,NaN
...,...,...,...,...,...,...
605,8464,億豐,14962.0,33000.0,NaN,NaN
607,6415,矽力-KY,13076.0,1000.0,NaN,NaN
635,2849,安泰銀,5000.0,3000.0,NaN,NaN
640,1476,儒鴻,3705.0,20000.0,NaN,NaN


## 外資買投信賣

In [19]:
#外資買投信賣
total[(total['外資買超']>0) &(total['投信賣超']<0)].reset_index(drop=True)


,代號,名稱,外資買超,投信買超,外資賣超,投信賣超
0,3481,群創,76115290.0,NaN,NaN,-352000.0
1,2409,友達,21746200.0,NaN,NaN,-30275000.0
2,2481,強茂,2982000.0,NaN,NaN,-1000000.0
3,2014,中鴻,2157200.0,NaN,NaN,-1048000.0
4,3006,晶豪科,1965826.0,NaN,NaN,-20000.0
5,2387,精元,1285000.0,NaN,NaN,-1069000.0
6,2027,大成鋼,1243295.0,NaN,NaN,-1842000.0
7,3413,京鼎,749000.0,NaN,NaN,-100000.0
8,2637,慧洋-KY,723581.0,NaN,NaN,-1041000.0
9,2456,奇力新,688000.0,NaN,NaN,-15000.0


## 外資賣，投信買

In [20]:
total[(total['外資賣超']<0) &(total['投信買超']>0)].reset_index(drop=True)

,代號,名稱,外資買超,投信買超,外資賣超,投信賣超
0,3037,欣興,NaN,1067000.0,-2942272.0,NaN
1,3189,景碩,NaN,645000.0,-3561548.0,NaN
2,0050,元大台灣50,NaN,548000.0,-1395367.0,NaN
3,2368,金像電,NaN,454000.0,-2705000.0,NaN
4,3017,奇鋐,NaN,404000.0,-420363.0,NaN
5,8081,致新,NaN,330000.0,-1403300.0,NaN
6,0056,元大高股息,NaN,300000.0,-793712.0,NaN
7,2006,東和鋼鐵,NaN,218000.0,-754000.0,NaN
8,8454,富邦媒,NaN,151000.0,-34348.0,NaN
9,5471,松翰,NaN,118000.0,-929900.0,NaN


## 外資投信同步賣

In [21]:
total[(total['外資賣超']<0) & (total['投信賣超']<0)].reset_index(drop=True)

,代號,名稱,外資買超,投信買超,外資賣超,投信賣超
0,2353,宏碁,NaN,NaN,-16793893.0,-19000.0
1,2103,台橡,NaN,NaN,-4872000.0,-1030000.0
2,1802,台玻,NaN,NaN,-4653700.0,-272000.0
3,2449,京元電子,NaN,NaN,-1653557.0,-486000.0
4,8016,矽創,NaN,NaN,-1407000.0,-610000.0
5,3044,健鼎,NaN,NaN,-852000.0,-190000.0
6,3665,貿聯-KY,NaN,NaN,-838200.0,-443000.0
7,2606,裕民,NaN,NaN,-762700.0,-241000.0
8,2308,台達電,NaN,NaN,-643734.0,-68000.0
9,6213,聯茂,NaN,NaN,-445000.0,-35000.0
